In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
%matplotlib inline
from fuzzywuzzy import fuzz
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12,4

In [2]:
def modelfit(alg,dtrain,dtest,predictiors, performCV = True, printFeatureImportance=True, cv_folds = 5):
    alg.fit(dtrain[predictors],dtrain['Is_Shortlisted'])
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    if performCV:
        cv_score = cross_validation.cross_val_score(alg,dtrain[predictors],dtrain['Is_Shortlisted'], cv = cv_folds, scoring = 'roc_auc')
        print "CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g"%(np.mean(cv_score),np.std(cv_score),np.min(cv_score),
                                                                        np.max(cv_score))
    print "Accuracy - %.4g" % metrics.accuracy_score(dtrain['Is_Shortlisted'],dtrain_predictions)
    print "AUC Score - %f" % metrics.roc_auc_score(dtrain['Is_Shortlisted'],dtrain_predprob)
    
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_,predictors).sort_values(ascending=False)
        feat_imp.plot(kind = 'bar', title = 'Feature Importance')
        plt.ylabel('Feature Importance Score')


In [3]:
data = pd.read_csv("C:/Users/user/Documents/data7.csv")
internship = pd.read_csv("C:/Users/user/Documents/Internship7.csv")
student = pd.read_csv("C:/Users/user/Documents/Student7.csv")
student_exp = pd.read_csv("C:/Users/user/Documents/Student_Exp7.csv")
data.drop('Unnamed: 0',axis=1,inplace = True)
internship.drop('Unnamed: 0',axis=1,inplace = True)
student.drop('Unnamed: 0',axis=1,inplace = True)
student_exp.drop('Unnamed: 0',axis=1,inplace = True)

In [10]:
#data['Preferred_location'] = data['Preferred_location'].apply(lambda x : 'NS' if pd.isnull(x) else x)
#internship['Skills_required'] = internship['Skills_required'].apply(lambda x : 'NS' if pd.isnull(x) else x)
#internship['skill_compress_str'] = internship['skill_compress_str'].apply(lambda x : 'NS' if pd.isnull(x) else x)
#datai = pd.merge(data,internship,how = 'left', on = 'Internship_ID')
#datais = pd.merge(datai,student,how = 'left', on = 'Student_ID')
#datais['Stipend1']= datais['Stipend1'].fillna(0)
#datais_dates = datais[['Student_ID','Earliest_Start_Date']]
#student_exp['Experience_Type'] = student_exp['Experience_Type'].fillna('NE')
#student_exp['Profile'] = student_exp['Profile'].fillna('NS')
student_exp.head()
#student_exp['exp_days'] = 0
#student_exp = student_exp[student_exp['Start.Date'].notnull()]
student_exp['Profile'] = student_exp[['Student_ID','Profile']].groupby(['Student_ID'])['Profile'].transform(lambda x: ','.join(x))
#student_exp['Experience_Type'] = student_exp[['Student_ID','Experience_Type']].groupby(['Student_ID'])['Experience_Type'].transform(lambda x: ','.join(x))
#student_exp_unique = student_exp[['Student_ID','Experience_Type','Profile']].drop_duplicates()
#dataise = pd.merge(datais,student_exp_unique,how='left',on = 'Student_ID')


,Student_ID,Experience_Type,Profile,Start.Date,End.Date,exp_days
0,7654321,job,"Software Engineering,Software Development",18-06-2013,21-06-2014,0
1,7654321,job,"Software Engineering,Software Development",01-07-2013,NaN,0
2,7668677,job,"Operations Process Consulting + Coding,Product...",06-05-2013,13-06-2014,0
3,7668677,job,"Operations Process Consulting + Coding,Product...",08-08-2011,29-03-2013,0
4,7654322,internship,Research Intern,02-06-2010,11-07-2010,0


In [11]:
dataise['avg_stipend'] = 0
dataise['avg_stipend'] = np.where(dataise['Stipend2'].isnull(),dataise['Stipend1'],(dataise['Stipend1']+dataise['Stipend2'])/2)
dataise.drop('Stipend2',axis=1,inplace=True)
dataise['percentage_pg'] = np.where(dataise['PG_scale'] == 4,(dataise['Performance_PG']+1)*20,np.where(dataise['PG_scale'] ==10 ,
                                                                                                   dataise['Performance_PG']*10,
                                                                                                   dataise['Performance_PG']))
dataise['percentage_ug'] = np.where(dataise['UG_Scale'] == 4,(dataise['Performance_UG']+1)*20,np.where(dataise['UG_Scale'] ==10 ,
                                                                                                   dataise['Performance_UG']*10,
                                                                                                   dataise['Performance_UG']))
dataise['student_stipend'] = np.where(dataise['Expected_Stipend']=='No Expectations',0 ,np.where(dataise['Expected_Stipend']==
                                                                                     '2-5K',2000,np.where(dataise['Expected_Stipend']==
                                                                                                         '5-10K',5000,10000)))
dataise['Stipend_low_match'] = np.where(dataise['Stipend1'] >= dataise['student_stipend'],'Yes','No')
dataise['Stipend_avg_match'] = np.where(dataise['avg_stipend'] >= dataise['student_stipend'],'Yes','No')
dataise.drop('Performance_PG',axis=1,inplace = True)
dataise.drop('Performance_UG',axis=1,inplace = True)
dataise['location_match'] = np.where(dataise['Preferred_location']=='NS','Yes',np.where(dataise['Preferred_location']==
                                                                                     dataise['Internship_Location'],'Yes','No'))


In [15]:
dataise['Internship_Profile'] = [nltk.word_tokenize(sent) for sent in dataise['Internship_Profile']]
dataise['Skills_required'] = [nltk.word_tokenize(sent) for sent in dataise['Skills_required']]
dataise['skill_compress_str'] = [nltk.word_tokenize(sent) for sent in dataise['skill_compress_str']]
dataise['Profile'] = [nltk.word_tokenize(sent.decode('utf8', 'ignore')) for sent in dataise['Profile']]

In [ ]:
dataise['degree_base'] = ''
for i in range(len(dataise)):
    dataise['Degree'][i] = str(dataise['Degree'][i])
    dataise['degree_base'][i] = np.where(re.search(r'^(u|b|B|U)',dataise['Degree'][i]),'UG',np.where(
            re.search(r'^(m|p|M|P)',dataise['Degree'][i]),'PG','Others'))
def str_replace(s):
    s= str(s)
    s= re.sub(r'^(b|B|U|u).*$',"UG",s)
    s= re.sub(r'^(p|P|M|m).*$',"PG",s)
    s= np.where(s=="UG","UG",np.where(s=="PG","PG","other"))
    return s    
dataise['degree_base'] = dataise['Degree'].apply(lambda x : str_replace(x))    


C:\Users\anoop.karnik\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\anoop.karnik\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
def common_elements(list1, list2):
    if isinstance(list1, basestring):
        if isinstance(list2,basestring):
            if list1==list2:
                a=['Match']
            else:
                a=[]
        else:
            for word in list2:
                if list1==word:
                    a=['Match']
                else:
                    a=[]
    else:
        a =  [element for element in list1 if element in list2]
    return len(a)

dataise['profile_common'] = 0
dataise['skill_common'] = 0
dataise['skill_common2'] = 0

for i in range(len(dataise)):
    dataise['profile_common'][i] = common_elements(dataise['Internship_Profile'][i],dataise['Profile'][i])
    dataise['skill_common'][i] = common_elements(dataise['Skills_required'][i],dataise['Profile'][i])
    dataise['skill_common'][i] = common_elements(dataise['skill_compress_str'][i],dataise['Profile'][i])

dataise['profile_common'] = dataise.apply(lamda x: common_elements(x['Internship_Profile'],x['Profile']),axis =1)
dataise['skill_common'] = dataise.apply(lamda x: common_elements(x['Skills_required'],x['Profile']),axis =1)
dataise['skill_common2'] = dataise.apply(lamda x: common_elements(x['skill_compress_str'],x['Profile']),axis =1)

for i in range(100):
    dataise['profile_common'][i] = common_elements(dataise['Internship_Profile'][i],dataise['Profile'][i])
    dataise['skill_common'][i] = common_elements(dataise['Skills_required'][i],dataise['Profile'][i])
    dataise['skill_common2'][i] = common_elements(dataise['skill_compress_str'][i],dataise['Profile'][i])
dataised = dataise[1:100]    
dataised[['profile_common','profile_common','Profile','Internship_Profile']]


C:\Users\anoop.karnik\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\anoop.karnik\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\anoop.karnik\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\anoop.karnik\AppData\Local\Continuum\Anaconda2\li

In [ ]:
data_new = [x for x in dataise.columns if x not in ['skill_compress_str','Degree','Stream','Internship_Location','Preferred_location',
                                                            'Profile','Internship_category','Internship_Profile','Experience_Type'
                                                  ,'Earliest_Start_Date','Start_Date','Internship_deadline','Skills_required' ]]
data_newest = dataise[data_new]
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
var_to_encod = ['Expected_Stipend','Is_Part_Time','Internship_Type',
                'Stipend_Type','Institute_Category','Year_of_graduation',
                 'PG_scale','UG_Scale','location_match','Stipend_low_match','Stipend_avg_match','Current_year','degree_base']
for col in var_to_encod:
    data_newest[col] = le.fit_transform(data_newest[col])
data_newest = pd.get_dummies(data_newest, columns=var_to_encod)

In [ ]:
train = data_newest.loc[data_newest['source'] == 'train']
test = data_newest.loc[data_newest['source']=='test']
train.drop('source',axis=1,inplace=True)
test.drop('Is_Shortlisted',axis=1,inplace=True)
train.to_csv("train4.csv")
test.to_csv("test4.csv")
#predictors = [x for x in train.columns if x not in ['Internship_ID','Student_ID','Is_Shortlisted']]
#gbm = GradientBoostingClassifier(random_state=143)
#modelfit(gbm,train,test,predictors)

In [ ]:
param_test1 = {'n_estimators': range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate = 0.1, min_samples_split =500,min_samples_leaf = 50,
                                                              max_depth = 8,max_features ='sqrt',subsample=0.8,random_state =10),
                      param_grid = param_test1,scoring='roc_auc',n_jobs=4,iid=False,cv=5)
gsearch1.fit(train[predictors],train['Is_Shortlisted'])
gsearch1.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train[predictors],train['Is_Shortlisted'])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test3 = {'min_samples_split':range(1000,2100,200), 'min_samples_leaf':range(30,71,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=9,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train[predictors],train['Is_Shortlisted'])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
modelfit(gsearch3.best_estimator_, train, predictors)

In [ ]:
param_test4 = {'max_features':range(7,20,2)}
gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=9, min_samples_split=1200, min_samples_leaf=60, subsample=0.8, random_state=10),
param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train[predictors],train['Is_Shortlisted'])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

In [ ]:
param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
gsearch5 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=9,min_samples_split=1200, min_samples_leaf=60, subsample=0.8, random_state=10,max_features=7),
param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(train[predictors],train['Is_Shortlisted']
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

In [ ]:
gbm_tuned_1 = GradientBoostingClassifier(learning_rate=0.05, n_estimators=120,max_depth=9, min_samples_split=1200,min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7)
modelfit(gbm_tuned_1, train, predictors)

In [ ]:
gbm_tuned_2 = GradientBoostingClassifier(learning_rate=0.01, n_estimators=600,max_depth=9, min_samples_split=1200,min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7)
modelfit(gbm_tuned_2, train, predictors)

In [ ]:
gbm_tuned_3 = GradientBoostingClassifier(learning_rate=0.005, n_estimators=1200,max_depth=9, min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7,
warm_start=True)
modelfit(gbm_tuned_3, train, predictors, performCV=False)

In [12]:
test['Is_Shortlisted'] = gbm.predict(test[predictors])
result = test[['Internship_ID','Student_ID','Is_Shortlisted']]
result.to_csv("result4.csv")


C:\Users\anoop.karnik\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [27]:
dataise['profile_common'] = dataise.apply(common_elements,args = (dataise[['Internship_Profile']],dataise[['Profile']]),axis=1)

'UG'

In [13]:
from functools import partial
def str_common_word(str1, str2):
    cnt =0
    str1 
    for word in str1:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

In [16]:

def common_elements(list1, list2):
    a =  [element for element in list1 if element in list 2]
    return len(a)
a= ['hello','killl','my','darling']
b= ['his','her','killl','hello','actor']
mydata = [{'subid' : 'B14-111', 'age': 75, 'fdg':1.78},
          {'subid' : 'B14-112', 'age': 22, 'fdg':1.56},]
c = a.apply(partial(common_elements,b))
#dataise['skill_common'] = dataise['Skills_required'].apply(partial(common_elements,dataise['Profile']))
#dataise['skill_common2'] = dataise['skill_compress_str'].apply(partial(common_elements,dataise['Profile']))

SyntaxError: invalid syntax (<ipython-input-16-5b94fb47a201>, line 3)

In [14]:
import functools
import operator

mydata = [{'a' : ['hello'], 'b': ['hello']},
          {'a' : 'kill', 'b': 'kka;jdk'},{'a' : 'hel', 'b': 'kll'},]
df = pd.DataFrame(mydata)


In [15]:
df['c'] = df['a'].apply(partial(str_common_word,df['b']))

AttributeError: 'list' object has no attribute 'find'

In [10]:
df

,a,b
0,[hello],[hello]
1,kill,kka;jdk
2,hel,kll


In [57]:
df

,a,b
0,"[hello, how, why]","[kill, hi, hello, how]"
1,kill,kka;jdk
2,hel,kll


In [6]:
def common_elements(list1, list2):
    if isinstance(list1, basestring):
        if isinstance(list2,basestring):
            if list1==list2:
                a=['Match']
            else:
                a=[]
        else:
            for word in list2:
                if list1==word:
                    a=['Match']
                else:
                    a=[]
    else:
        a =  [element for element in list1 if element in list2]
    return len(a)

mydata = [{'a' : ['hello','kill','hel'], 'b': ['hel','hello','kill']},
          {'a' : ['kill','ad'],'b': ['kka;jdk','adf']},{'a' : 'hell', 'b': ['kll','yeah']},]
df = pd.DataFrame(mydata)
import functools
import operator
from functools import partial

In [9]:
#df['c'] = df['a'].apply(partial(common_elements,df['b']))

df['c'] = df['a'].apply(partial(common_elements,df['b']))
    

TypeError: 'in <string>' requires string as left operand, not list

In [52]:
for a in data_new:
    print dataise[a].value_counts()

6188     1183
5487      902
5908      796
9284      768
10055     718
4579      711
6158      677
8858      677
9197      620
6121      611
8444      598
10428     595
9850      569
6876      562
10070     553
7845      547
9935      543
7051      531
10342     526
5603      524
4823      524
6992      500
4944      487
5662      481
8740      472
7393      470
10761     463
9292      462
5087      455
5003      448
         ... 
6178        1
10622       1
9743        1
7758        1
10795       1
4731        1
8975        1
5036        1
8619        1
6572        1
6607        1
6422        1
7945        1
8267        1
6190        1
9768        1
6979        1
5614        1
5010        1
8681        1
9330        1
4724        1
9329        1
5750        1
9556        1
10279       1
7887        1
9365        1
10764       1
7148        1
Name: Internship_ID, dtype: int64
7706394     428
7693536     259
7708160     240
7693439     237
7654664     225
7671428     216
7672358     213


In [46]:
.value_counts()

UG                                           142937
PG                                            47463
12th                                            946
Integrated B.Tech and M.Tech                    589
Integrated B.Tech and MBA                       559
Diploma                                         408
Graduation                                      311
IMCA                                            200
Dual Degree (5 Years)                           195
Integrated MBA                                  193
LLB                                             163
Integrated MCA                                  150
Chartered Accountant (CA)                       112
Integrated Masters                              104
Architecture                                    104
No                                              102
Hsc                                              96
Dual Degree                                      96
Graduate                                         93
Not a gradua

In [ ]:
dataise.apply(lambda x: sum(x.isnull()))

Internship_ID                       0
Student_ID                          0
Earliest_Start_Date                 0
Expected_Stipend                    0
Minimum_Duration                    0
Preferred_location                  0
Is_Part_Time                        0
Is_Shortlisted                 107428
source                              0
Internship_Profile                  0
Skills_required                     0
Internship_Type                     0
Internship_Location                 0
Internship_category                 0
No_of_openings                      0
Stipend_Type                        0
Stipend1                            0
Internship_deadline                 0
Start_Date                          0
Internship_Duration.Months.         0
PR                                  0
UI                                  0
Marketing                           0
Media                               0
Social                              0
Design                              0
Web         